Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



Nesse experimento revisitamos o problema da mochila, esse problema é bastante interessante de ser estudado por tratar de um problema de maximização dado que queremos encontrar o maior valor possível, mas com o acréscimo de uma restrição, nesse caso peso.

Tendo isso em mente, houve, nesse caso, a mudança da função de seleção de `selecao_torneio_min` para `solucao_roleta_max`.

Podemos ver, como outra novidade, a função `funcao_objetivo_pop_mochila`e ela tem por objetivo computar a `função objetivo` de um candidato na mochila retornando o valor total dos itens inseridos aplicando também determinada penalidade para quando o peso é excedido.

Além disso, a implementação da função `computa_mochila`, perimitiu com maior facilidade calcularmos o valor total e o peso total das mochilas. 

Vale ainda ressaltar que esse experimento, assim como o anterior, também trata-se de um problema NP difícil, onde só é possível obter a resposta de verdade se todas as possibilidades fossem testadas.


In [ ]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 15
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
    "teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "energético": {
        "peso": 2,
        "valor": 1500,
    },
    "docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
# ordenando a lista
# poderia ser feito de outras maneira
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

Para dar início ao experimentos definimos valores e computamos alguns dos parâmetros, bem como ordenamos a lista, salvando essa ordem dos nomes, o que nos será muito útil posteriormente.

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [4]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf") # levando em consideração que é um problema de maximização
melhor_individuo_ja_visto = [0] * NUM_OBJETOS # solução trivial para o problema 

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO: # não mostrar coisas q n resolvem o problema
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

Maior valor: 11050 | Peso: 14.1
Maior valor: 12400 | Peso: 13.1
Maior valor: 16550 | Peso: 14.1

Você deve pegar os seguintes itens:
+ Quadrinho super raro do Aranha-Homem da vida real
+ energético
+ mesa dobrável para laptop
+ tablet
+ teclado musical
+ vinil falsificado da volta do One Direction

Com isso, sua mochila terá o valor de 16550 dinheiros e peso de 14.1 unidades de massa.


Para construção de todo o raciocínio, foi ainda preciso lembrar de realizar algumas leves alterações em comparação com o experimento anterior.

Uma dessas alterações foi no parâmetro de fitness, por exemplo, que teve seu sinal trocado tendo em vista que agora tratamos de programas de maximização,o mesmo aconteceu com alguns dos sinais utilizados na implementação do `hall da fama`&#x1F31F;!

## Conclusão



Tendo em vista que o nosso principal objetivo era obter soluções para o problema da mochila em que os valores nela contidos fossem os maiores possíveis e o peso limite não fossem ultrapassados, podemos dizer que nossos resultados aparentam ser consistentes.

Podemos dizer também que a implementação de penalidades para conjuntos de mochila que ultrapassassem o peso limite bom muito bem sucedida e será mais uma boa saída para termos na nossa caixa de ferramentas de algoritmos genéticos.

Essa série de experimentos foi bastante interessante de ser discutida e permitiu não só uma aprendizagem sobre algoritmos genéticos, mas também sobre organização e implementação de código, gostei bastante!

## Playground

